In [2]:
import pandas as pd

In [9]:
from harvai.data import get_clean_preproc_data

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Text2TextGenerationPipeline
from harvai.data import preprocessing_user_input

In [2]:
data = get_clean_preproc_data()

In [3]:
model_name = 'lincoln/barthez-squadFR-fquad-piaf-question-generation'

loaded_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
loaded_tokenizer = AutoTokenizer.from_pretrained(model_name)




In [32]:
questions = []
articles = []
for article in range(0,1148):
    
    if(len(data.iloc[article,2])) > 1020: 
        text = data.iloc[article,2][0:1020] 
    else:    
        text = data.iloc[article,2]
        
    inputs = loaded_tokenizer(text, return_tensors='pt')

    out = loaded_model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        num_beams=5,
        num_return_sequences=5,
        length_penalty=20
    )


    for question in out:
        questions.append(loaded_tokenizer.decode(question, skip_special_tokens=True))
        articles.append(article)


In [33]:
df_questions =pd.DataFrame(columns=['questions','id'])
df_questions['questions'] =questions 
df_questions['id'] =articles 

In [39]:
df_questions.to_csv('../raw_data/generated_questions_dataset.csv')

In [4]:
df_questions =pd.read_csv('../raw_data/generated_questions_dataset.csv')

In [7]:
df_questions.drop(columns='Unnamed: 0',inplace=True)

In [15]:
df_questions['questions_preproc'] = df_questions['questions'].apply(lambda x : preprocessing_user_input(x))

In [17]:
df_questions.to_csv('../raw_data/generated_questions_dataset.csv')

In [3]:
df_questions =pd.read_csv('../raw_data/generated_questions_dataset.csv')

In [8]:
df_questions.dropna(inplace=True)

In [9]:
df_questions.to_csv('../raw_data/generated_questions_dataset.csv')